# 95-891 Homework 5 – AutoML

## Blaine Perry
## Andrew ID: blainep
Due Arpil 14th, 2022


In [1]:
import pandas as pd
import numpy as np

from math import sin
from math import pi
from numpy import arange
from numpy import vstack
from numpy import argmax
from numpy import asarray
from numpy.random import normal
from numpy.random import random
from scipy.stats import norm
from sklearn.gaussian_process import GaussianProcessRegressor
from warnings import catch_warnings
from warnings import simplefilter
from matplotlib import pyplot
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import r2_score
import itertools

from time import time
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

#### Read in the Data

In [2]:
# Read in the data
df = pd.read_csv('hw5.csv')

# select relevant features
# irrelevant features are ignored
selected_features = ['#Followers',
                     '#Friends',
                     '#Retweets',
                     '#Favorites',
                     'Positive_sentiment',
                     'Negative_sentiment',
                     'Post_hour']

selected_df = df[selected_features]

# split the data into train and test
train_df = selected_df.iloc[:80000]
test_df = selected_df.iloc[80000:]


X_train, y_train = train_df.loc[:, train_df.columns != '#Retweets'], train_df['#Retweets']
X_test, y_test = test_df.loc[:, test_df.columns != '#Retweets'], test_df['#Retweets']

# standardize the data
scalar = StandardScaler().fit(X_train)
X_train = scalar.transform(X_train)
X_test = scalar.transform(X_test)

In [3]:
print('X_train shape: ', X_train.shape)
print('y_train shape: ',y_train.shape)
print('X_test shape: ',X_test.shape)
print('y_test shape: ',y_test.shape)

X_train shape:  (80000, 6)
y_train shape:  (80000,)
X_test shape:  (20000, 6)
y_test shape:  (20000,)


In [4]:
# Utility function to report best scores
# code from https://scikit-learn.org/stable/auto_examples/model_selection/plot_randomized_search.html
def report(results, n_top=1):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results["rank_test_score"] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print(
                "Mean validation score: {0:.3f} (std: {1:.3f})".format(
                    results["mean_test_score"][candidate],
                    results["std_test_score"][candidate],
                )
            )
            print("Parameters: {0}".format(results["params"][candidate]))
            print("")



In [20]:
# create the model and its hyperparameters
clf = MLPRegressor(random_state=0)
param_dist = {'hidden_layer_sizes': [(10,), (20,), (30,), (40,), (50,),
                                     (60,), (70,), (80,), (90,), (100,)],
              'learning_rate_init': [0.001, 0.01, 0.1]}

n_iter_search = 20

In [21]:
# ###########################################################################
# #Conduct randomsearch here with the predefined range.
# 1-2 lines.
random_search = RandomizedSearchCV(clf, param_dist, n_jobs=-1)
##############################################################################

start = time()
random_search.fit(X_train, y_train)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))

RandomizedSearchCV took 75.58 seconds for 20 candidates parameter settings.


In [26]:
##############################################################################
# Print the highest R^2 on X_train.
# Use the corresponding model to predict on X_test and get the R^2 score.

clf_best = random_search.best_estimator_

y_pred_train = clf_best.predict(X_train)
y_pred_test = clf_best.predict(X_test)

r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)

print('Highest x_train R^2: ' + str(r2_train))
print('Highest x_test R^2: ' + str(r2_test))
report(random_search.cv_results_)
##############################################################################

Highest x_train R^2: 0.7854397246934642
Highest x_test R^2: 0.6920379648057144
Model with rank: 1
Mean validation score: 0.695 (std: 0.104)
Parameters: {'learning_rate_init': 0.1, 'hidden_layer_sizes': (50,)}



In [8]:
##############################################################################
# Conduct GridSearch here with the predefined range.
# 1-2 lines.
grid_search = GridSearchCV(clf, param_dist, n_jobs=-1)
##############################################################################

start = time()
grid_search.fit(X_train, y_train)
print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
      % (time() - start, len(grid_search.cv_results_['params'])))

GridSearchCV took 232.68 seconds for 33 candidate parameter settings.


C:\Users\blain\anaconda3\envs\95891\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [27]:
##############################################################################
# Print the highest R^2 on X_train.
# Use the corresponding model to predict on X_test and get the R^2 score.
clf_best_grid = grid_search.best_estimator_

y_pred_train = clf_best_grid.predict(X_train)
y_pred_test = clf_best_grid.predict(X_test)

r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)

print('Highest x_train R^2: ' + str(r2_train))
print('Highest x_test R^2: ' + str(r2_test))
report(grid_search.cv_results_)
##############################################################################

Highest x_train R^2: 0.782634091221836
Highest x_test R^2: 0.7993652664129686
Model with rank: 1
Mean validation score: 0.704 (std: 0.182)
Parameters: {'hidden_layer_sizes': (10,), 'learning_rate_init': 0.001}



In [10]:
# subset the data
X_train_new, X_valid = X_train[:70000, :], X_train[70000:, :]
y_train_new, y_valid = y_train[:70000], y_train[70000:]


In [11]:
# objective function
# return the R^2 score on the validation set by parameters
def objective(X_train, y_train, X_valid, y_valid, parameters):
    clf = MLPRegressor(hidden_layer_sizes=parameters[0],
                       learning_rate_init=parameters[1],
                       random_state=0)
    clf.fit(X_train, y_train)
    y_valid_predicted = clf.predict(X_valid)
    return r2_score(y_valid, y_valid_predicted)

In [12]:
# surrogate or approximation for the objective function
def surrogate(model, X):
##############################################################################
# This function should return model's (Gaussian Process model) prediction value
# on X. Here X is a hyperparameter combination, e.g., [20, 0.01].
# The function returns both the predicted value and the uncertainty (std).
    mu, std = model.predict(X, return_std=True)
    return mu, std
##############################################################################

In [13]:
def acquisition_UCB(Xsamples, model):
    # calculate mean and stdev via surrogate function
    mu, std = surrogate(model, Xsamples)
    ##############################################################################
    return (mu + std) / 2
    # you could simply return the average of the predicted value and the uncertainty.
##############################################################################

In [14]:
# optimize the acquisition function
def opt_acquisition(X, model):
    # random search, generate random samples
    n_layers = np.random.randint(1, 100, 100)
    learning_rates = np.random.uniform(0, 1, 100)
    params = list(itertools.product(n_layers, learning_rates))
    Xsamples = [list(ele) for ele in params]

    # calculate the acquisition function for each sample
    scores = acquisition_UCB(Xsamples, model)

    # locate the index of the largest scores
    ix = np.argmax(scores)
    return Xsamples[ix]

In [15]:
# create some initial points for GP
params = [[10, 0.01],
          [20, 0.1],
          [50, 0.01],
          [100, 0.001],
          [100, 0.01]]
y_params = [objective(X_train_new, y_train_new, X_valid, y_valid, param)
            for param in params]

##############################################################################
# Here you should initialize a Gaussian Process model and fit with initial params
# and the corresponding y_params

model = GaussianProcessRegressor()
model.fit(params, y_params)
##############################################################################

GaussianProcessRegressor()

In [16]:

#start a start time for the process
start = time()

# perform the optimization process
for i in range(50):
    # select the next point to sample
    x = opt_acquisition(params, model)
    # sample the point
    actual = objective(X_train_new, y_train_new, X_valid, y_valid, x)
    # summarize the finding
    est, _ = surrogate(model, np.asarray(x).reshape(1, -1))

    # add the data to the dataset
    params.append(x)
    y_params.append(actual)

    #print(x, actual)

    ##############################################################################
    # # update the Gaussian Process model
    # you need one line here to update the GP model with new observations.
    model.fit(params, y_params)
##############################################################################

print("Bayesian Optimization took %.2f seconds."
      % (time() - start))

[51, 0.5911551220358146] 0.5668907320268495
[99, 0.004021887818007497] 0.5978890489888182
[11, 0.613524039815785] 0.5399126492247341
[49, 0.950877967788447] 0.48447557918176465
[50, 0.9827423152362325] 0.4662777947753477
[10, 0.9987479885922803] -0.19558086331453772
[12, 0.006145513030104355] 0.5868439328771695


C:\Users\blain\anaconda3\envs\95891\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


[11, 0.0007637205740924635] 0.538070210397946
[12, 0.979239620267569] 0.13294134593230655
[21, 0.8998405888120525] 0.36472401981430447
[19, 0.9967909515819313] 0.45948455782232767


C:\Users\blain\anaconda3\envs\95891\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


[48, 0.0003661726465133963] 0.5675292906388374
[18, 0.04562035253902008] 0.5863834960935004
[17, 0.8618714391479438] 0.13563651858695525
[47, 0.9107867046841598] 0.31948473175119774
[19, 0.020187172621119576] 0.5916845979746659


C:\Users\blain\anaconda3\envs\95891\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


[9, 0.0013416094431786263] 0.5984076903259485
[14, 0.02117051425850025] 0.5921785287315473
[13, 0.01241237258878225] 0.5946529334272366
[52, 0.0029682218688358297] 0.5939627508577099
[53, 0.891252891490782] 0.4349325103605509
[49, 0.01023606482754058] 0.5951749967069391
[15, 0.9881763226911041] 0.12098326245968583
[54, 0.003423461313310616] 0.5954277714807257
[55, 0.7867862240702225] 0.4912215652088586
[56, 0.03735975451662732] 0.5424115068676219
[57, 0.9599610790848776] 0.3274604804522211
[7, 0.007218206661937954] 0.590246540410288
[7, 0.9972091128840588] 0.36268726519735994
[8, 0.0039683401781627214] 0.5971641414459141
[5, 0.05833097577401403] 0.5893663692711514
[5, 0.9963800741658632] 0.02837841103153238
[4, 0.005305989023028701] 0.5897182575551014
[55, 0.005003531701888164] 0.5948701696590124
[2, 0.004554911091262603] -5.283656418253457e-05
[97, 0.0012800370293796215] 0.5954188058690373
[96, 0.6962397832636313] 0.3372855331117035
[58, 0.0013822377683427867] 0.5976642948289332
[59, 

In [17]:
# best result
ix = argmax(y_params)
print('Best R^2 for X_train: ', y_params[ix])

clf = MLPRegressor(hidden_layer_sizes=params[ix][0],
                   learning_rate_init=params[ix][1],
                   random_state=0)

clf.fit(X_train, y_train)

test_prediction = clf.predict(X_test)
print('Best R^2 for X_test', r2_score(y_test, test_prediction))

Best R^2 for X_train:  0.5984076903259485
Best R^2 for X_test 0.7994995400755596


C:\Users\blain\anaconda3\envs\95891\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


### 8. Compare RandomSearch and GridSearch with Bayesian optimization, what’s your observation regarding both accuracy (train and test R^2 score) and the elapsed time?
###### RandomSearch was the fastest of the three, with GridSearch and Bayesian optimization being far slower.  For accuracy, the Bayesian optimization produced a model which was slightly more accurate on the test data, but was worse on training data.  The RandomSearch found the worst model in terms of test and training accuracy.  This makes sense as it does not properly evaluate the test space, and only takes random samples.